In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [2]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [3]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var='Global_active_power', dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

## Dataset Household Power Consumption 

In [17]:
filepath = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/household_power_consumption.csv'
data = pd.read_csv(filepath, sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
data = data.loc[0: : 30] # 30 minutes
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()


/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
30,2.720,0.000,235.06,11.6,0.0,0.0,17.0
60,3.452,0.000,235.20,15.2,0.0,1.0,17.0
90,4.298,0.000,232.39,18.4,0.0,1.0,16.0
120,3.262,0.052,232.64,14.0,0.0,0.0,17.0


In [18]:
data['Global_active_power'] = pd.to_numeric(data['Global_active_power'],errors='coerce')
data['Global_reactive_power'] = pd.to_numeric(data['Global_reactive_power'],errors='coerce')
data['Voltage'] = pd.to_numeric(data['Voltage'],errors='coerce')
data['Global_intensity'] = pd.to_numeric(data['Global_intensity'],errors='coerce')
data['Sub_metering_1'] = pd.to_numeric(data['Sub_metering_1'],errors='coerce')
data['Sub_metering_2'] = pd.to_numeric(data['Sub_metering_2'],errors='coerce')
data['Sub_metering_3'] = pd.to_numeric(data['Sub_metering_3'],errors='coerce')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68308 entries, 0 to 2075250
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Global_active_power    68308 non-null  float64
 1   Global_reactive_power  68308 non-null  float64
 2   Voltage                68308 non-null  float64
 3   Global_intensity       68308 non-null  float64
 4   Sub_metering_1         68308 non-null  float64
 5   Sub_metering_2         68308 non-null  float64
 6   Sub_metering_3         68308 non-null  float64
dtypes: float64(7)
memory usage: 4.2 MB


### Dataset Household: columns

### Dataset Household: Preprocessing, Lag = 1, t+1

In [19]:
# frame as supervised learning
data = series_to_supervised_miso(data, 1, 1,endog_var='Global_active_power')
data.head()

,Global_active_power(t-1),Global_reactive_power(t-1),Voltage(t-1),Global_intensity(t-1),Sub_metering_1(t-1),Sub_metering_2(t-1),Sub_metering_3(t-1),Global_active_power(t)
30,4.216,0.418,234.84,18.4,0.0,1.0,17.0,2.720
60,2.720,0.000,235.06,11.6,0.0,0.0,17.0,3.452
90,3.452,0.000,235.20,15.2,0.0,1.0,17.0,4.298
120,4.298,0.000,232.39,18.4,0.0,1.0,16.0,3.262
150,3.262,0.052,232.64,14.0,0.0,0.0,17.0,3.214


## Model: PcaWeightedMVFTS - PCA-MVFTS

In [20]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 3,
                       endogen_variable = 'Global_active_power(t-1)',
                       n_part = 50)

In [21]:
result = {
     "window": [],
     "rmse": [],
     "mae": [],
     "mape": [],
     "smape": [],
     "nrmse":[]
}

num_fuzzy_rules = {
    "window": [],
    "qtd": []
}

tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
        
        data_train = ttrain.loc[:,'Global_active_power(t-1)':'Sub_metering_3(t-1)']
        data_test = ttest.loc[:,'Global_reactive_power(t-1)':'Global_active_power(t)']
         
        #ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
        data_test.rename(columns = {'Global_active_power(t)': 'Global_active_power(t-1)'}, inplace = True)
        
        print('-' * 20)
        print(f'training window {(ct)}')
        model, pca_reduced_train = pca_wmvfts.run_train_model(data_train)
        forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,data_test)
        
        num_fuzzy_rules["window"].append(ct)
        num_fuzzy_rules["qtd"].append(len(model))
        
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        
        #Teste e cálculo dos erros da previsão  
        #valor use Global_reactive_power(t-1) é o Global_reactive_power(t) --> label igual por causa do modelo 
        rmse = Measures.rmse(pca_reduced_test['Global_active_power(t-1)'],forecast)
        mape = Measures.mape(pca_reduced_test['Global_active_power(t-1)'],forecast)
        smape = Measures.smape(pca_reduced_test['Global_active_power(t-1)'],forecast)
        
        forecast = pd.DataFrame(forecast)
        forecast.fillna(forecast.mean(),inplace=True)
        forecast = np.array(forecast).reshape(-1)
        mae = mean_absolute_error(pca_reduced_test['Global_active_power(t-1)'], forecast)
        
        nrmse = round(cal_nrmse(rmse, pca_reduced_test['Global_active_power(t-1)']),3)
        
        print(f'Results: {(rmse,mae,mape,smape,nrmse)}')
        
#         # Grafico de comparação Original x Previsto para cada janela
#         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#         ax.plot(pca_reduced_test['Global_active_power(t-1)'], label='Original')
#         ax.plot(forecast, label='Forecast')
#         handles, labels = ax.get_legend_handles_labels()
#         lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#         plt.show()
        
        result["rmse"].append(round(rmse,3))
        result["mape"].append(round(mape,3))
        result["smape"].append(round(smape,3))
        result["mae"].append(round(mae,3))
        result["nrmse"].append(round(nrmse,3))
        result["window"].append(ct)
        
measures = pd.DataFrame(result)
rules = pd.DataFrame(num_fuzzy_rules)



/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


--------------------
training window 0
[ 23:41:46] getting statistics
Results: (0.10935971313062534, 0.046165337359855625, 8.130436607444723, 3.3174791495858584, 0.015)
--------------------
training window 2276


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:41:50] getting statistics
Results: (0.09507553169050045, 0.03128516490102911, 4.130685747689725, 1.9013435160224856, 0.012)
--------------------
training window 4552


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


...

[ 23:43:41] getting statistics
Results: (0.05522467198897568, 0.01715250548622147, 8.1502986588354, 2.8490024835101044, 0.03)
--------------------
training window 63728


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:43:45] getting statistics
Results: (0.06449421427344428, 0.03075506497494304, 4.62172558331087, 1.9397286586125537, 0.011)
--------------------
training window 66004


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:43:49] getting statistics
Results: (0.003827906207110826, 0.015660988755951444, 0.23856788614684613, 0.11536881284118466, 0.0)


In [23]:
measures

,window,rmse,mae,mape,smape,nrmse
0,0,0.109,0.046,8.130,3.317,0.015
1,2276,0.095,0.031,4.131,1.901,0.012
2,4552,0.097,0.038,12.680,4.483,0.016
3,6828,0.024,0.005,2.328,0.908,0.004
4,9104,0.046,0.006,0.464,0.265,0.010
5,11380,0.054,0.013,2.053,0.979,0.008
6,13656,0.058,0.027,0.345,0.161,0.007
7,15932,0.086,0.045,3.449,1.618,0.011
8,18208,0.127,0.060,15.648,5.496,0.021
9,20484,0.057,0.008,0.692,0.305,0.010


In [13]:
#measures.to_csv (r'win_pca_wmvfts_hcp_1min.csv', index = False, header=True)

In [24]:
measures.to_csv (r'win_pca_wmvfts_hcp_30min.csv', index = False, header=True)

### Mean Statistics PCA-WMVFTS:

In [25]:
final_result = {
     "rmse": [],
     "mae": [],
     "mape": [],
     "smape": [],
     "nrmse":[],
     "rmse_std": [],
     "mae_std": [],
     "mape_std": [],
     "smape_std": [],
     "nrmse_std": [],
     "rules":[]
}

final_result["rmse"].append(round(statistics.mean(measures['rmse']),3))
final_result["mae"].append(round(statistics.mean(measures['mae']),3))
final_result["mape"].append(round(statistics.mean(measures['mape']),3))
final_result["smape"].append(round(statistics.mean(measures['smape']),3))
final_result["nrmse"].append(round(statistics.mean(measures['nrmse']),3))

final_result["rmse_std"].append(round(statistics.stdev(measures['rmse']),3))
final_result["mape_std"].append(round(statistics.stdev(measures['mape']),3))
final_result["smape_std"].append(round(statistics.stdev(measures['smape']),3))
final_result["mae_std"].append(round(statistics.stdev(measures['mae']),3))
final_result["nrmse_std"].append(round(statistics.stdev(measures['nrmse']),3))

final_result["rules"].append(round(statistics.mean(rules.loc[:,'qtd']),3))

        
final_measures_pca = pd.DataFrame(final_result)

print("Mean Statistics PCA-WMVFTS (test): ")
final_measures_pca

Mean Statistics PCA-WMVFTS (test): 


,rmse,mae,mape,smape,nrmse,rmse_std,mae_std,mape_std,smape_std,nrmse_std,rules
0,0.069,0.025,5.076,1.925,0.013,0.038,0.017,4.228,1.474,0.009,6555.567


In [16]:
#final_measures_pca.to_csv (r'pca_wmvfts_hpc_1min.csv', index = False, header=True)

In [26]:
final_measures_pca.to_csv (r'pca_wmvfts_hpc_30min.csv', index = False, header=True)

### Extras 

In [12]:
# pca_reduced_test

In [13]:
# print(model)

In [14]:
# train = sample_first_prows(data,0.75)
# test = data.iloc[max(train.index):]
# y_test = data.iloc[max(train.index):]['Appliances_t+1'].values

In [15]:
# pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
#                        endogen_variable = 'Appliances_t+1',
#                        n_part = 50)

In [16]:
# model, pca_reduced_train = pca_wmvfts.run_train_model(train)
# forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,test)

In [17]:
#model.mvflrg

In [18]:
# forecast_1 = model.predict(pca_reduced_test)

In [19]:
# Measures.rmse(y_test,forecast)

In [20]:
# Measures.rmse(forecast_1,y_test)

In [21]:
# Measures.rmse(y_test,forecast_self)

In [22]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
# ax.plot(y_test, label='Original')
# ax.plot(forecast, label='Forecast')
# handles, labels = ax.get_legend_handles_labels()
# lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
# plt.show()


In [23]:
# diff = data.loc[:,'lights':'Tdewpoint']
# diff = diff.diff()
# diff['Appliances'] = data['Appliances']
# data = diff.dropna()